In [1]:
from pynq import Overlay
import time
import ctypes
import cv2
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from ipywidgets import *

def usleep(microseconds):
    # 使用time.sleep实现微秒级别的睡眠
    time.sleep(microseconds / 1000000)

## 综合1（gly+zxy+wyf的IP综合）

In [2]:
zonghe1 = Overlay("/home/xilinx/pynq/overlays/base/zonghe1/zonghe1.bit")

In [3]:
?zonghe1

## GLY图像处理IP，预先读取图片版本

In [2]:
# base = Overlay("/home/xilinx/pynq/overlays/base/TEST/CCC.bit")

In [3]:
# ?base

In [4]:
mem = zonghe1.axi_bram_ctrl_0

In [5]:
?mem

In [6]:
data = mem.read(4)
print("%08X" % data)

0FFDFE9C


In [7]:
binary_num = bin(data)[2:].zfill(32)
print(binary_num)

00001111111111011111111010011100


In [8]:
def gen_arr(h, w, mem):
    i = 1
    string = ''
    while i <= h * w / 32:
        data = mem.read(i * 4)
        binary_num = bin(data)[2:].zfill(32)
        string += binary_num
        i += 1
    shift = [c - 1 for c in range(32, w+1, 32)]
    img_array = np.zeros((h, w), dtype=np.uint8)
    for i in range(h):
        for j in range(w):
            if string[i * w + j] == '0':
                img_array[i, j] = 0
            else:
                img_array[i, j] = 255
        for item in shift:
            if item != shift[-1]:
                img_array[i, item] = img_array[i, item+32]
            else:
                img_array[i, item] = 0
    return img_array

In [9]:
w, h = 1024, 384
start = time.time()
img = gen_arr(h, w, mem)
end = time.time() - start
print(end)
# plt.imshow(img)
# plt.show()

2.337505340576172


## I2C

In [23]:
# i2c_master = Overlay("/home/xilinx/pynq/overlays/base/wyf/i2c.bit")

In [26]:
# ?i2c_master

In [38]:
i2c = zonghe1.i2c_wyf_0
SLV_REG = [0, 4, 8, 12, 16, 20, 24, 28, 32, 36, 40, 44, 48, 52]

In [42]:
def i2c_read():
    i2c.write(SLV_REG[0], 0x00000000)
    time.sleep(1)
    i2c.write(SLV_REG[0], 0x00000001)
    i2c.write(SLV_REG[1], 0x00000001)
    i2c.write(SLV_REG[2], 0x00000050)
    i2c.write(SLV_REG[2], 0x00000029)
    i2c.write(SLV_REG[3], 0x000000C1)
    time.sleep(1)
    print("%u\n" % i2c.read(SLV_REG[10]))

In [43]:
i2c_read()

170



## SPI

In [2]:
# spi_master = Overlay("/home/xilinx/pynq/overlays/base/zxy-spi/spi.bit")

In [3]:
# ?spi_master

In [4]:
spi = spi_master.spi_oled111_0
SPI_REG = [0, 4, 8, 12, 16, 20, 24, 28, 32, 36, 40, 44, 48, 52]
OLED_CMD = 0x00
OLED_DATA = 0x01
SPI_SIZE = 16
Max_Column = 128

In [5]:
Hzk=[
[0x04,0x04,0x04,0xE4,0x24,0x24,0x25,0x26,0x24,0x24,0x24,0xE4,0x04,0x04,0x04,0x00],
[0x00,0x40,0x20,0x1B,0x02,0x42,0x82,0x7E,0x02,0x02,0x02,0x0B,0x10,0x60,0x00,0x00],
[0x04,0x24,0x44,0x84,0x64,0x9C,0x40,0x30,0x0F,0xC8,0x08,0x08,0x28,0x18,0x00,0x00],
[0x10,0x08,0x06,0x01,0x82,0x4C,0x20,0x18,0x06,0x01,0x06,0x18,0x20,0x40,0x80,0x00],
[0x40,0x40,0x42,0xCC,0x00,0x00,0xFC,0x04,0x02,0x00,0xFC,0x04,0x04,0xFC,0x00,0x00],
[0x00,0x40,0x20,0x1F,0x20,0x40,0x4F,0x44,0x42,0x40,0x7F,0x42,0x44,0x43,0x40,0x00],
[0x40,0x40,0x42,0x44,0x58,0xC0,0x40,0x7F,0x40,0xC0,0x50,0x48,0x46,0x40,0x40,0x00],
[0x80,0x80,0x40,0x20,0x18,0x07,0x00,0x00,0x00,0x3F,0x40,0x40,0x40,0x40,0x78,0x00],
[0x00,0xF8,0x00,0x00,0xFF,0x40,0x20,0x18,0x0F,0x18,0x68,0x08,0x08,0x08,0x08,0x00],
[0x00,0x1F,0x00,0x00,0xFF,0x00,0x00,0x7F,0x21,0x21,0x3F,0x21,0x21,0x7F,0x00,0x00]
]

F8X16=[
  0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,
  0x00,0x00,0x00,0xF8,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x33,0x30,0x00,0x00,0x00,
  0x00,0x10,0x0C,0x06,0x10,0x0C,0x06,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,
  0x40,0xC0,0x78,0x40,0xC0,0x78,0x40,0x00,0x04,0x3F,0x04,0x04,0x3F,0x04,0x04,0x00,
  0x00,0x70,0x88,0xFC,0x08,0x30,0x00,0x00,0x00,0x18,0x20,0xFF,0x21,0x1E,0x00,0x00,
  0xF0,0x08,0xF0,0x00,0xE0,0x18,0x00,0x00,0x00,0x21,0x1C,0x03,0x1E,0x21,0x1E,0x00,
  0x00,0xF0,0x08,0x88,0x70,0x00,0x00,0x00,0x1E,0x21,0x23,0x24,0x19,0x27,0x21,0x10,
  0x10,0x16,0x0E,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,
  0x00,0x00,0x00,0xE0,0x18,0x04,0x02,0x00,0x00,0x00,0x00,0x07,0x18,0x20,0x40,0x00,
  0x00,0x02,0x04,0x18,0xE0,0x00,0x00,0x00,0x00,0x40,0x20,0x18,0x07,0x00,0x00,0x00,
  0x40,0x40,0x80,0xF0,0x80,0x40,0x40,0x00,0x02,0x02,0x01,0x0F,0x01,0x02,0x02,0x00,
  0x00,0x00,0x00,0xF0,0x00,0x00,0x00,0x00,0x01,0x01,0x01,0x1F,0x01,0x01,0x01,0x00,
  0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x80,0xB0,0x70,0x00,0x00,0x00,0x00,0x00,
  0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x01,0x01,0x01,0x01,0x01,0x01,0x01,
  0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x30,0x30,0x00,0x00,0x00,0x00,0x00,
  0x00,0x00,0x00,0x00,0x80,0x60,0x18,0x04,0x00,0x60,0x18,0x06,0x01,0x00,0x00,0x00,
  0x00,0xE0,0x10,0x08,0x08,0x10,0xE0,0x00,0x00,0x0F,0x10,0x20,0x20,0x10,0x0F,0x00,
  0x00,0x10,0x10,0xF8,0x00,0x00,0x00,0x00,0x00,0x20,0x20,0x3F,0x20,0x20,0x00,0x00,
  0x00,0x70,0x08,0x08,0x08,0x88,0x70,0x00,0x00,0x30,0x28,0x24,0x22,0x21,0x30,0x00,
  0x00,0x30,0x08,0x88,0x88,0x48,0x30,0x00,0x00,0x18,0x20,0x20,0x20,0x11,0x0E,0x00,
  0x00,0x00,0xC0,0x20,0x10,0xF8,0x00,0x00,0x00,0x07,0x04,0x24,0x24,0x3F,0x24,0x00,
  0x00,0xF8,0x08,0x88,0x88,0x08,0x08,0x00,0x00,0x19,0x21,0x20,0x20,0x11,0x0E,0x00,
  0x00,0xE0,0x10,0x88,0x88,0x18,0x00,0x00,0x00,0x0F,0x11,0x20,0x20,0x11,0x0E,0x00,
  0x00,0x38,0x08,0x08,0xC8,0x38,0x08,0x00,0x00,0x00,0x00,0x3F,0x00,0x00,0x00,0x00,
  0x00,0x70,0x88,0x08,0x08,0x88,0x70,0x00,0x00,0x1C,0x22,0x21,0x21,0x22,0x1C,0x00,
  0x00,0xE0,0x10,0x08,0x08,0x10,0xE0,0x00,0x00,0x00,0x31,0x22,0x22,0x11,0x0F,0x00,
  0x00,0x00,0x00,0xC0,0xC0,0x00,0x00,0x00,0x00,0x00,0x00,0x30,0x30,0x00,0x00,0x00,
  0x00,0x00,0x00,0x80,0x00,0x00,0x00,0x00,0x00,0x00,0x80,0x60,0x00,0x00,0x00,0x00,
  0x00,0x00,0x80,0x40,0x20,0x10,0x08,0x00,0x00,0x01,0x02,0x04,0x08,0x10,0x20,0x00,
  0x40,0x40,0x40,0x40,0x40,0x40,0x40,0x00,0x04,0x04,0x04,0x04,0x04,0x04,0x04,0x00,
  0x00,0x08,0x10,0x20,0x40,0x80,0x00,0x00,0x00,0x20,0x10,0x08,0x04,0x02,0x01,0x00,
  0x00,0x70,0x48,0x08,0x08,0x08,0xF0,0x00,0x00,0x00,0x00,0x30,0x36,0x01,0x00,0x00,
  0xC0,0x30,0xC8,0x28,0xE8,0x10,0xE0,0x00,0x07,0x18,0x27,0x24,0x23,0x14,0x0B,0x00,
  0x00,0x00,0xC0,0x38,0xE0,0x00,0x00,0x00,0x20,0x3C,0x23,0x02,0x02,0x27,0x38,0x20,
  0x08,0xF8,0x88,0x88,0x88,0x70,0x00,0x00,0x20,0x3F,0x20,0x20,0x20,0x11,0x0E,0x00,
  0xC0,0x30,0x08,0x08,0x08,0x08,0x38,0x00,0x07,0x18,0x20,0x20,0x20,0x10,0x08,0x00,
  0x08,0xF8,0x08,0x08,0x08,0x10,0xE0,0x00,0x20,0x3F,0x20,0x20,0x20,0x10,0x0F,0x00,
  0x08,0xF8,0x88,0x88,0xE8,0x08,0x10,0x00,0x20,0x3F,0x20,0x20,0x23,0x20,0x18,0x00,
  0x08,0xF8,0x88,0x88,0xE8,0x08,0x10,0x00,0x20,0x3F,0x20,0x00,0x03,0x00,0x00,0x00,
  0xC0,0x30,0x08,0x08,0x08,0x38,0x00,0x00,0x07,0x18,0x20,0x20,0x22,0x1E,0x02,0x00,
  0x08,0xF8,0x08,0x00,0x00,0x08,0xF8,0x08,0x20,0x3F,0x21,0x01,0x01,0x21,0x3F,0x20,
  0x00,0x08,0x08,0xF8,0x08,0x08,0x00,0x00,0x00,0x20,0x20,0x3F,0x20,0x20,0x00,0x00,
  0x00,0x00,0x08,0x08,0xF8,0x08,0x08,0x00,0xC0,0x80,0x80,0x80,0x7F,0x00,0x00,0x00,
  0x08,0xF8,0x88,0xC0,0x28,0x18,0x08,0x00,0x20,0x3F,0x20,0x01,0x26,0x38,0x20,0x00,
  0x08,0xF8,0x08,0x00,0x00,0x00,0x00,0x00,0x20,0x3F,0x20,0x20,0x20,0x20,0x30,0x00,
  0x08,0xF8,0xF8,0x00,0xF8,0xF8,0x08,0x00,0x20,0x3F,0x00,0x3F,0x00,0x3F,0x20,0x00,
  0x08,0xF8,0x30,0xC0,0x00,0x08,0xF8,0x08,0x20,0x3F,0x20,0x00,0x07,0x18,0x3F,0x00,
  0xE0,0x10,0x08,0x08,0x08,0x10,0xE0,0x00,0x0F,0x10,0x20,0x20,0x20,0x10,0x0F,0x00,
  0x08,0xF8,0x08,0x08,0x08,0x08,0xF0,0x00,0x20,0x3F,0x21,0x01,0x01,0x01,0x00,0x00,
  0xE0,0x10,0x08,0x08,0x08,0x10,0xE0,0x00,0x0F,0x18,0x24,0x24,0x38,0x50,0x4F,0x00,
  0x08,0xF8,0x88,0x88,0x88,0x88,0x70,0x00,0x20,0x3F,0x20,0x00,0x03,0x0C,0x30,0x20,
  0x00,0x70,0x88,0x08,0x08,0x08,0x38,0x00,0x00,0x38,0x20,0x21,0x21,0x22,0x1C,0x00,
  0x18,0x08,0x08,0xF8,0x08,0x08,0x18,0x00,0x00,0x00,0x20,0x3F,0x20,0x00,0x00,0x00,
  0x08,0xF8,0x08,0x00,0x00,0x08,0xF8,0x08,0x00,0x1F,0x20,0x20,0x20,0x20,0x1F,0x00,
  0x08,0x78,0x88,0x00,0x00,0xC8,0x38,0x08,0x00,0x00,0x07,0x38,0x0E,0x01,0x00,0x00,
  0xF8,0x08,0x00,0xF8,0x00,0x08,0xF8,0x00,0x03,0x3C,0x07,0x00,0x07,0x3C,0x03,0x00,
  0x08,0x18,0x68,0x80,0x80,0x68,0x18,0x08,0x20,0x30,0x2C,0x03,0x03,0x2C,0x30,0x20,
  0x08,0x38,0xC8,0x00,0xC8,0x38,0x08,0x00,0x00,0x00,0x20,0x3F,0x20,0x00,0x00,0x00,
  0x10,0x08,0x08,0x08,0xC8,0x38,0x08,0x00,0x20,0x38,0x26,0x21,0x20,0x20,0x18,0x00,
  0x00,0x00,0x00,0xFE,0x02,0x02,0x02,0x00,0x00,0x00,0x00,0x7F,0x40,0x40,0x40,0x00,
  0x00,0x0C,0x30,0xC0,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x01,0x06,0x38,0xC0,0x00,
  0x00,0x02,0x02,0x02,0xFE,0x00,0x00,0x00,0x00,0x40,0x40,0x40,0x7F,0x00,0x00,0x00,
  0x00,0x00,0x04,0x02,0x02,0x02,0x04,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,
  0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x80,0x80,0x80,0x80,0x80,0x80,0x80,0x80,
  0x00,0x02,0x02,0x04,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,
  0x00,0x00,0x80,0x80,0x80,0x80,0x00,0x00,0x00,0x19,0x24,0x22,0x22,0x22,0x3F,0x20,
  0x08,0xF8,0x00,0x80,0x80,0x00,0x00,0x00,0x00,0x3F,0x11,0x20,0x20,0x11,0x0E,0x00,
  0x00,0x00,0x00,0x80,0x80,0x80,0x00,0x00,0x00,0x0E,0x11,0x20,0x20,0x20,0x11,0x00,
  0x00,0x00,0x00,0x80,0x80,0x88,0xF8,0x00,0x00,0x0E,0x11,0x20,0x20,0x10,0x3F,0x20,
  0x00,0x00,0x80,0x80,0x80,0x80,0x00,0x00,0x00,0x1F,0x22,0x22,0x22,0x22,0x13,0x00,
  0x00,0x80,0x80,0xF0,0x88,0x88,0x88,0x18,0x00,0x20,0x20,0x3F,0x20,0x20,0x00,0x00,
  0x00,0x00,0x80,0x80,0x80,0x80,0x80,0x00,0x00,0x6B,0x94,0x94,0x94,0x93,0x60,0x00,
  0x08,0xF8,0x00,0x80,0x80,0x80,0x00,0x00,0x20,0x3F,0x21,0x00,0x00,0x20,0x3F,0x20,
  0x00,0x80,0x98,0x98,0x00,0x00,0x00,0x00,0x00,0x20,0x20,0x3F,0x20,0x20,0x00,0x00,
  0x00,0x00,0x00,0x80,0x98,0x98,0x00,0x00,0x00,0xC0,0x80,0x80,0x80,0x7F,0x00,0x00,
  0x08,0xF8,0x00,0x00,0x80,0x80,0x80,0x00,0x20,0x3F,0x24,0x02,0x2D,0x30,0x20,0x00,
  0x00,0x08,0x08,0xF8,0x00,0x00,0x00,0x00,0x00,0x20,0x20,0x3F,0x20,0x20,0x00,0x00,
  0x80,0x80,0x80,0x80,0x80,0x80,0x80,0x00,0x20,0x3F,0x20,0x00,0x3F,0x20,0x00,0x3F,
  0x80,0x80,0x00,0x80,0x80,0x80,0x00,0x00,0x20,0x3F,0x21,0x00,0x00,0x20,0x3F,0x20,
  0x00,0x00,0x80,0x80,0x80,0x80,0x00,0x00,0x00,0x1F,0x20,0x20,0x20,0x20,0x1F,0x00,
  0x80,0x80,0x00,0x80,0x80,0x00,0x00,0x00,0x80,0xFF,0xA1,0x20,0x20,0x11,0x0E,0x00,
  0x00,0x00,0x00,0x80,0x80,0x80,0x80,0x00,0x00,0x0E,0x11,0x20,0x20,0xA0,0xFF,0x80,
  0x80,0x80,0x80,0x00,0x80,0x80,0x80,0x00,0x20,0x20,0x3F,0x21,0x20,0x00,0x01,0x00,
  0x00,0x00,0x80,0x80,0x80,0x80,0x80,0x00,0x00,0x33,0x24,0x24,0x24,0x24,0x19,0x00,
  0x00,0x80,0x80,0xE0,0x80,0x80,0x00,0x00,0x00,0x00,0x00,0x1F,0x20,0x20,0x00,0x00,
  0x80,0x80,0x00,0x00,0x00,0x80,0x80,0x00,0x00,0x1F,0x20,0x20,0x20,0x10,0x3F,0x20,
  0x80,0x80,0x80,0x00,0x00,0x80,0x80,0x80,0x00,0x01,0x0E,0x30,0x08,0x06,0x01,0x00,
  0x80,0x80,0x00,0x80,0x00,0x80,0x80,0x80,0x0F,0x30,0x0C,0x03,0x0C,0x30,0x0F,0x00,
  0x00,0x80,0x80,0x00,0x80,0x80,0x80,0x00,0x00,0x20,0x31,0x2E,0x0E,0x31,0x20,0x00,
  0x80,0x80,0x80,0x00,0x00,0x80,0x80,0x80,0x80,0x81,0x8E,0x70,0x18,0x06,0x01,0x00,
  0x00,0x80,0x80,0x80,0x80,0x80,0x80,0x00,0x00,0x21,0x30,0x2C,0x22,0x21,0x30,0x00,
  0x00,0x00,0x00,0x00,0x80,0x7C,0x02,0x02,0x00,0x00,0x00,0x00,0x00,0x3F,0x40,0x40,
  0x00,0x00,0x00,0x00,0xFF,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0xFF,0x00,0x00,0x00,
  0x00,0x02,0x02,0x7C,0x80,0x00,0x00,0x00,0x00,0x40,0x40,0x3F,0x00,0x00,0x00,0x00,
  0x00,0x06,0x01,0x01,0x02,0x02,0x04,0x04,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00
]


In [6]:
def OLED_WR_Byte(spi, dat, cmd):
    reg_val = 0x00
    if cmd == OLED_CMD:
        reg_val = 0x00
    else:
        reg_val = 0x01
    spi.write(SPI_REG[2], reg_val)
    spi.write(SPI_REG[0], 0)
    spi.write(SPI_REG[1], dat)
    spi.write(SPI_REG[0], 1)
    usleep(1)

In [7]:
def OLED_Set_Pos(spi, x, y):
    OLED_WR_Byte(spi, 0xb0 + y, OLED_CMD)
    OLED_WR_Byte(spi, ((x&0xf0)>>4)|0x10,OLED_CMD)
    OLED_WR_Byte(spi, (x&0x0f)|0x01,OLED_CMD)

In [8]:
def OLED_ShowCHinese(spi, x, y, no):
    OLED_Set_Pos(spi, x, y)
    for t in range(16):
        OLED_WR_Byte(spi, Hzk[2*no][t], OLED_DATA)
    OLED_Set_Pos(spi, x,y+1);
    for t in range(16):
        OLED_WR_Byte(spi, Hzk[2*no+1][t], OLED_DATA)

In [9]:
def OLED_ShowChar(spi, x, y, chr):
    c = ord(chr) - ord(' ')
    if x > Max_Column-1:
        x = 0
        y = y + 2
    if SPI_SIZE == 16:
        OLED_Set_Pos(spi, x, y)
        for i in range(8):
            OLED_WR_Byte(spi, F8X16[c*16+i], OLED_DATA)
        OLED_Set_Pos(spi, x,y+1)
        for i in range(8):
            OLED_WR_Byte(spi, F8X16[c*16+i+8], OLED_DATA)

In [10]:
def OLED_ShowPlate(spi, x, y, num:list):
    OLED_ShowCHinese(spi, x, y, 0)
    x += 16
    j = 0
    while num[j] != '\0':
        OLED_ShowChar(spi, x, y, num[j])
        x += 8
        j += 1

In [11]:
def OLED_Clear(spi):
    for i in range(8):
        OLED_WR_Byte(spi, 0xb0+i, OLED_CMD)
        OLED_WR_Byte(spi, 0x00, OLED_CMD)
        OLED_WR_Byte(spi, 0x10, OLED_CMD)
        for n in range(128):
            OLED_WR_Byte(spi, 0, OLED_DATA)

In [12]:
def OLED_INIT(spi):
    OLED_WR_Byte(spi, 0xAE,OLED_CMD)
    OLED_WR_Byte(spi, 0x00,OLED_CMD)
    OLED_WR_Byte(spi, 0x10,OLED_CMD)
    OLED_WR_Byte(spi, 0x40,OLED_CMD)
    OLED_WR_Byte(spi, 0x81,OLED_CMD)
    OLED_WR_Byte(spi, 0xCF,OLED_CMD)
    OLED_WR_Byte(spi, 0xA1,OLED_CMD)
    OLED_WR_Byte(spi, 0xC8,OLED_CMD)
    OLED_WR_Byte(spi, 0xA6,OLED_CMD)
    OLED_WR_Byte(spi, 0xA8,OLED_CMD)
    OLED_WR_Byte(spi, 0x3f,OLED_CMD)
    OLED_WR_Byte(spi, 0xD3,OLED_CMD)
    OLED_WR_Byte(spi, 0x00,OLED_CMD)
    OLED_WR_Byte(spi, 0xd5,OLED_CMD)
    OLED_WR_Byte(spi, 0x80,OLED_CMD)
    OLED_WR_Byte(spi, 0xD9,OLED_CMD)
    OLED_WR_Byte(spi, 0xF1,OLED_CMD)
    OLED_WR_Byte(spi, 0xDA,OLED_CMD)
    OLED_WR_Byte(spi, 0x12,OLED_CMD)
    OLED_WR_Byte(spi, 0xDB,OLED_CMD)
    OLED_WR_Byte(spi, 0x40,OLED_CMD)
    OLED_WR_Byte(spi, 0x20,OLED_CMD)
    OLED_WR_Byte(spi, 0x02,OLED_CMD)
    OLED_WR_Byte(spi, 0x8D,OLED_CMD)
    OLED_WR_Byte(spi, 0x14,OLED_CMD)
    OLED_WR_Byte(spi, 0xA4,OLED_CMD)
    OLED_WR_Byte(spi, 0xA6,OLED_CMD)
    OLED_WR_Byte(spi, 0xAF,OLED_CMD)

    OLED_WR_Byte(spi, 0xAF,OLED_CMD)
    OLED_Clear(spi)
    OLED_Set_Pos(spi, 0,0)

In [13]:
def welcome_oled(spi):
    for i in range(1, 5):
        OLED_ShowCHinese(spi, 0+16*(i-1),2, i)

In [14]:
def OLED_main(spi):
    OLED_INIT(spi)
    plate = [c for c in "CBA105\0"]
    OLED_ShowPlate(spi, 0, 0, plate)
    welcome_oled(spi)

In [15]:
OLED_main(spi)

## uart

In [2]:
# uart_master =  Overlay("/home/xilinx/pynq/overlays/base/zxy-uart/uart.bit")

In [4]:
# ?uart_master

In [3]:
uart = zonghe1.axi_uartlite_0

In [6]:
?uart

In [5]:
def convertCharToGBK(c):
    if c >= 'A' and c <= 'Z':
        return 0x41 + (ord(c) - ord('A'))
    elif c >= '0' and c <= '9':
        return 0x30 + (ord(c) - ord('0'))
    return 0x00;


In [17]:
jing = [0xBE, 0xA9, convertCharToGBK('C'), convertCharToGBK('B'), convertCharToGBK('G'), convertCharToGBK('1'), convertCharToGBK('0'), convertCharToGBK('5')]
for i in range(8):
    uart.write(4, jing[i])